<a href="https://colab.research.google.com/github/tinosai/tinosai.github.io/blob/master/CreditCardFraud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
!echo '{"username":"tino91","key":"fc01868fec179ecbb5efeeef5522ca43"}'>kaggle.json
!mv kaggle.json /root/.kaggle/
!kaggle datasets download -d mlg-ulb/creditcardfraud


 92% 61.0M/66.0M [00:00<00:00, 47.1MB/s]
100% 66.0M/66.0M [00:00<00:00, 79.5MB/s]


In [4]:
!unzip creditcardfraud.zip

Archive:  creditcardfraud.zip
  inflating: creditcard.csv          


In [0]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


import os
import pandas as pd
import numpy as np


# In[2]:


df=pd.read_csv('creditcard.csv')


# In[3]:


df.head()


# In[4]:


#Time is not needed
df=df.drop(["Time"],axis=1)
df.head()


# In[5]:


#since the great majority of the transactions are non fraudulent, let's look for the first 10 fraudulent transactions
df[df["Class"]==1].head(10).drop(["Class"],axis=1)


# In[73]:


#let's reshuffle the dataset
df=df.sample(frac=1)


# In[74]:


#lets create train, validation, test sets
p=0.8
kiri=int(p*df.shape[0])
train_X,val_X,test_X=np.array(df.iloc[:kiri,:-1]),np.array(df.iloc[kiri::2,:-1]),np.array(df.iloc[kiri+1::2,:-1])
train_Y,val_Y,test_Y=np.array(df.iloc[:kiri,-1]),np.array(df.iloc[kiri::2,-1]),np.array(df.iloc[kiri+1::2,-1])


# In[77]:


from collections import Counter
train_Y_count=Counter(train_Y)
val_Y_count=Counter(val_Y)
test_Y_count=Counter(test_Y)


# In[78]:


print("In train set: {} negative and {} positive".format(train_Y_count[0],train_Y_count[1]))
print("In val set: {} negative and {} positive".format(val_Y_count[0],val_Y_count[1]))
print("In test set: {} negative and {} positive".format(test_Y_count[0],test_Y_count[1]))

print("Train Set, Positive/Negative: {} %".format(train_Y_count[1]/train_Y_count[0]*100))
print("Val Set, Positive/Negative: {} %".format(val_Y_count[1]/val_Y_count[0]*100))
print("Test Set, Positive/Negative: {} %".format(test_Y_count[1]/test_Y_count[0]*100))


# The percentage of positives in the validation and test set is higher than in the training set. As a result, this would guarantee more stringent requirements on accuracy.

# In[255]:


#first of all, we need to normalize the inputs
#mean vector
mean_vector=train_X.mean(axis=0)
#std vector
std_vector=train_X.std(axis=0)


# In[256]:


normalized_train_X,normalized_val_X,normalized_test_X=(train_X-mean_vector)/std_vector,(val_X-mean_vector)/std_vector,(test_X-mean_vector)/std_vector


# In[526]:


import torch
from torch.utils.data import TensorDataset, DataLoader


# In[527]:


#TensorDataset creation
train_data=TensorDataset(torch.from_numpy(normalized_train_X),torch.from_numpy(train_Y))
val_data=TensorDataset(torch.from_numpy(normalized_val_X),torch.from_numpy(val_Y))
test_data=TensorDataset(torch.from_numpy(normalized_test_X),torch.from_numpy(test_Y))


# In[528]:


#class imbalance treatment
num_classes=[train_Y_count[0],train_Y_count[1]]
weights=1./torch.tensor(num_classes,dtype=float)
sample_weights=weights[train_Y]
sampler=torch.utils.data.WeightedRandomSampler(weights=sample_weights,num_samples=len(sample_weights),replacement=True)


# In[771]:


#creating the loaders
batch_size=50001 #was 100
train_loader=DataLoader(train_data,sampler=sampler,batch_size=batch_size)
val_loader=DataLoader(val_data,shuffle=True,batch_size=batch_size)
test_loader=DataLoader(test_data,shuffle=False,batch_size=batch_size)


# In[910]:


train_iter=iter(val_loader)
inputs,outputs=train_iter.next()
print(inputs)


# In[983]:


import torch.nn as nn
import torch.nn.functional as F


# In[984]:


class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.fc1=nn.Linear(29,20)
        self.fc2=nn.Linear(20,10)
        self.fc3=nn.Linear(10,3)
        self.fc4=nn.Linear(3,1)
        #self.fc1=nn.Linear(29,15)
        #self.fc2=nn.Linear(15,8)
        #self.fc3=nn.Linear(8,3)
        #self.fc4=nn.Linear(3,1)
        
        
    def forward(self,x):
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=F.relu(self.fc3(x))
        x=self.fc4(x)        
        return x


# In[985]:


model=Net()


# In[986]:


model


# In[987]:


criterion=nn.BCEWithLogitsLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.001,weight_decay=1e-4)
#0.0005, 1e-3, F1=0.485 @50 iter
#0.0001, 1e-2, F1=0.205 @50iter 
#0.00025, 1e-3, F1=0.3786 @50iter
#0.0003, 5e-4, F1=0.52 @50iter
#0.0003, 2.5e-4, F1=0.5970 @50iter
#0.00025,2.5e-4, F1=0.457 @50iter
#0.00035,2.5e-4, F1=0.5674 @50iter
#0.0003,2e-4, F1=0.5672 @50iter
#0.0003, 3.0e-4, F1=0.5909 @50iter
#0.00035, 2.5e-4, F1= @50iter

#0.00015, 2.5e-4, F1=0.644 @500iter
#0.00015, 1e-4, F1=0.7723 @500iter
#0.00015, 0.75e-4, F1=0.7047 @500iter
#0.0001, 1e-4, F1=0.7358 @500iter
#0.00015, 1e-4, F1=0.65117 @50iter #optimizer beta 0.8
#0.00015, 3e-4, F1=0.6190 @50iter #optimizer beta 0.8
#0.00015, 1e-4, F1=0.6722 @50iter
#0.0005, 1e-4, F1= @50iter


# In[988]:


print(optimizer)


# In[989]:

model.cuda()

def train(n_epochs,loaders,model,optimizer,criterion):
    list_train_loss=[]
    list_valid_loss=[]
    for epoch in range(1,n_epochs+1):
        train_loss=0.0
        valid_loss=0.0
        model.train()
        for batch_idx,(data,target) in enumerate(loaders["train"]):
            optimizer.zero_grad()
            data,target=data.cuda(),target.cuda()
            output=model(data.float())
            loss=criterion(output.squeeze(),target.float())
            loss.backward()
            optimizer.step()
            train_loss=train_loss+1/(batch_idx+1)*(loss.item()-train_loss)
        list_train_loss.append(train_loss)
        print("At {} epoch, Training Loss: {} ".format(epoch,train_loss))
        model.eval()
        for batch_idx,(data,target) in enumerate(loaders["valid"]):
            data,target=data.cuda(),target.cuda()
            output=model(data.float())
            loss=criterion(output.squeeze(),target.float())
            valid_loss=valid_loss+1/(batch_idx+1)*(loss.item()-valid_loss)
        list_valid_loss.append(valid_loss)
        print("At {} epoch, Validation Loss: {} ".format(epoch,valid_loss))
    return list_train_loss, list_valid_loss


# In[990]:


#loader wrapping
loaders={'train':train_loader, 'valid':val_loader}


# In[991]:


train_loss,valid_loss=train(150,loaders,model,optimizer,criterion) 


# In[ ]:


import matplotlib.pyplot as plt


# In[ ]:


plt.plot(train_loss)
plt.plot(valid_loss)


# In[ ]:




In train set: 227451 negative and 394 positive
In val set: 28438 negative and 43 positive
In test set: 28426 negative and 55 positive
Train Set, Positive/Negative: 0.17322412299792042 %
Val Set, Positive/Negative: 0.15120613263942612 %
Test Set, Positive/Negative: 0.19348483782452686 %
tensor([[ 0.8085, -0.8916, -0.4972,  ..., -0.2322, -0.0387,  0.6520],
        [-1.4035, -1.7850,  1.6955,  ...,  0.4594, -1.1748,  0.3357],
        [-0.3642, -0.2338,  0.9832,  ...,  0.8763,  0.3724, -0.3571],
        ...,
        [ 0.5733, -0.5754,  0.2896,  ..., -0.0249,  0.1054,  0.2208],
        [-0.2265,  0.3950,  0.6745,  ...,  0.9162,  0.5195, -0.3495],
        [-0.3750,  1.1196, -0.8985,  ...,  0.6489,  0.3527, -0.3314]],
       dtype=torch.float64)
Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0.0001
)
At 1 epoch, Training Loss: 0.6910328388214111 
At 1 epoch, Validation Loss: 0.9237477779388428 
At 2 epoch, Training Loss: 0.67

In [12]:
outputs=[]
targets=[]
for data,target in test_loader:
    data,target=data.cuda(),target.cuda()
    output=model(data.float())
    output=output.cpu()
    output=list(np.where(output<0,0,1)[:])
    target=list(target.cpu().numpy())
    outputs+=output
    targets+=target


# In[ ]:


ts=0
for i in outputs:
    ts+=i


# In[ ]:


outputs_of_model=np.stack(outputs,axis=0).squeeze().astype(dtype=int)
targets_of_model=np.array(targets).astype(dtype=int)


# In[ ]:


TN=np.array(targets_of_model[outputs_of_model==0]==0).sum()
FP=np.array(targets_of_model[outputs_of_model==1]==0).sum()
FN=np.array(targets_of_model[outputs_of_model==0]==1).sum()
TP=np.array(targets_of_model[outputs_of_model==1]==1).sum()


# In[ ]:


print("True Positives: {}".format(TP))
print("True Negatives: {}".format(TN))
print("False Positives: {}".format(FP))
print("False Negatives: {}".format(FN))


# In[ ]:


precision=TP/(TP+FP)
recall=TP/(TP+FN)
print(precision)
print(recall)


# In[ ]:


F1=2*precision*recall/(precision+recall)


# In[ ]:


print(F1)



True Positives: 41
True Negatives: 28034
False Positives: 404
False Negatives: 2
0.09213483146067415
0.9534883720930233
0.1680327868852459
